In [1]:
import pandas as pd
import mrmr
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from statistics import fmean
from sklearn.svm import SVC
from numpy import mean

In [2]:
circles, half_kernel, moons, spiral, two_gaussians33, two_gaussians42, breast_cancer =  pd.read_csv("SampleDatasets/circles0.3.csv"), pd.read_csv("SampleDatasets/halfkernel.csv"), pd.read_csv("SampleDatasets/moons1.csv"), pd.read_csv("SampleDatasets/spiral1.csv"), pd.read_csv("SampleDatasets/twogaussians33.csv"),  pd.read_csv("SampleDatasets/twogaussians42.csv"), pd.read_csv("SampleDatasets/Breastcancer.csv")

In [3]:
def evaluation_function(model_object, x_test, y_test, multi_class=False):
    if not multi_class:
        tn, fp, fn, tp = confusion_matrix(y_test, model_object.predict(x_test)).ravel()
        specificity = tn / (tn + fp)
        sensitivity = tp / (tp + fn)
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        ppv = tp / (tp + fp)
        npv = tn / (fn + tn)
    else:
        list_of_matrix = multilabel_confusion_matrix(y_test, model_object.predict(x_test))
        specificity, sensitivity, accuracy, ppv, npv = list(), list(), list(), list(), list()
        for matrix in list_of_matrix:
            tn, fp, fn, tp = matrix.ravel()
            specificity.append(tn / (tn + fp))
            sensitivity.append(tp / (tp + fn))
            accuracy.append((tp + tn) / (tp + tn + fp + fn))
            ppv.append(tp / (tp + fp))
            npv.append(tn / (fn + tn))
        specificity, sensitivity, accuracy, ppv, npv = fmean(specificity), fmean(sensitivity), fmean(accuracy), fmean(ppv), fmean(npv)

    return specificity, sensitivity, accuracy, ppv, npv

In [4]:
def print_results(model_object, x_test, y_test, multi_class=False):

    evaluation_results = evaluation_function(model_object, x_test, y_test, multi_class)
    print("specificity: {} \nsensitivity: {} \naccuracy: {} \nppv: {} \nnpv: {}".format(evaluation_results[0], evaluation_results[1], evaluation_results[2], evaluation_results[3], evaluation_results[4]))

In [5]:
def perform_kfold(model_object, training_data, training_labels, n_splits=10):

    cv = KFold(n_splits=n_splits, random_state=1, shuffle=True)
    scores = cross_val_score(model_object, training_data, training_labels, scoring='accuracy', cv=cv, n_jobs=-1)
    return mean(scores)

In [6]:
def svm_grid_search(training_data, training_labels, n_folds=10):

    parameter_dict = {'kernel': ('linear', 'poly', 'rbf'), 'degree': range(2, 6), 'gamma': ('scale', 'auto'), 'C': [0.001, 0.01, 0.1, 1]}
    new_classifier = SVC()
    grid = GridSearchCV(new_classifier, param_grid=parameter_dict, cv=n_folds, verbose=1)
    grid.fit(training_data, training_labels)

    return grid.best_params_, grid.best_score_

# Circle0.3

In [7]:
circles_x_train, circles_x_test, circles_y_train, circles_y_test = train_test_split(circles.iloc[:, 0:2], circles['label'], test_size=0.3, random_state=42)

### Decision Tree on Circle0.3 Dataset

In [8]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(circles_x_train, circles_y_train)

DecisionTreeClassifier()

In [9]:
decision_tree.tree_.max_depth

5

In [10]:
print_results(decision_tree, circles_x_test, circles_y_test)

specificity: 0.9803921568627451 
sensitivity: 0.9795918367346939 
accuracy: 0.98 
ppv: 0.9795918367346939 
npv: 0.9803921568627451


In [11]:
perform_kfold(decision_tree, circles_x_train, circles_y_train)

0.99

### Random Forest on Circle0.3

In [12]:
random_forest = RandomForestClassifier()
random_forest.fit(circles_x_train, circles_y_train)

RandomForestClassifier()

In [13]:
print_results(random_forest, circles_x_test, circles_y_test)

specificity: 1.0 
sensitivity: 0.9727891156462585 
accuracy: 0.9866666666666667 
ppv: 1.0 
npv: 0.9745222929936306


In [14]:
perform_kfold(random_forest, circles_x_train, circles_y_train)

0.9957142857142858

### Multi Layer Perceptron on Circle0.3

In [15]:
perceptron_parameters = {'hidden_layer_sizes': (10,10),
                         'activation': 'relu',
                         'solver': 'adam',
                         'max_iter': 400,
                         'learning_rate': 'adaptive'}

perceptron = MLPClassifier(**perceptron_parameters)
perceptron.fit(circles_x_train, circles_y_train)

MLPClassifier(hidden_layer_sizes=(10, 10), learning_rate='adaptive',
              max_iter=400)

In [16]:
print_results(perceptron, circles_x_test, circles_y_test)

specificity: 1.0 
sensitivity: 1.0 
accuracy: 1.0 
ppv: 1.0 
npv: 1.0


In [17]:
perform_kfold(perceptron, circles_x_train, circles_y_train)

0.9985714285714286

# Halfkernel

In [18]:
half_kernel_x_train, half_kernel_x_test, half_kernel_y_train, half_kernel_y_test = train_test_split(half_kernel.iloc[:, 0:2], half_kernel['label'], test_size=0.3, random_state=42)

### Decision Tree on Halfkernel Dataset

In [19]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(half_kernel_x_train, half_kernel_y_train)

DecisionTreeClassifier()

In [20]:
decision_tree.tree_.max_depth

5

In [21]:
print_results(decision_tree, half_kernel_x_test, half_kernel_y_test)

specificity: 0.9866666666666667 
sensitivity: 0.9933333333333333 
accuracy: 0.99 
ppv: 0.9867549668874173 
npv: 0.9932885906040269


In [22]:
perform_kfold(decision_tree, half_kernel_x_train, half_kernel_y_train)

0.9957142857142858

### Random Forest on Halfkernel

In [23]:
random_forest = RandomForestClassifier()
random_forest.fit(half_kernel_x_train, half_kernel_y_train)

RandomForestClassifier()

In [24]:
print_results(random_forest, half_kernel_x_test, half_kernel_y_test)

specificity: 0.9866666666666667 
sensitivity: 0.9933333333333333 
accuracy: 0.99 
ppv: 0.9867549668874173 
npv: 0.9932885906040269


In [25]:
perform_kfold(random_forest, half_kernel_x_train, half_kernel_y_train)

0.9971428571428571

### Multi Layer Perceptron on Halfkernel

In [26]:
perceptron_parameters = {'hidden_layer_sizes': (10,10),
                         'activation': 'relu',
                         'solver': 'adam',
                         'max_iter': 400,
                         'learning_rate': 'adaptive'}

perceptron = MLPClassifier(**perceptron_parameters)
perceptron.fit(half_kernel_x_train, half_kernel_y_train)

MLPClassifier(hidden_layer_sizes=(10, 10), learning_rate='adaptive',
              max_iter=400)

In [27]:
print_results(perceptron, half_kernel_x_test, half_kernel_y_test)

specificity: 1.0 
sensitivity: 1.0 
accuracy: 1.0 
ppv: 1.0 
npv: 1.0


In [28]:
perform_kfold(perceptron, half_kernel_x_train, half_kernel_y_train)

1.0

# Moons1

In [29]:
moons_x_train, moons_x_test, moons_y_train, moons_y_test = train_test_split(moons.iloc[:, 0:2], moons['label'], test_size=0.3, random_state=42)

### Decision Tree on Moons1

In [30]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(moons_x_train, moons_y_train)

DecisionTreeClassifier()

In [31]:
decision_tree.tree_.max_depth

7

In [32]:
print_results(decision_tree, moons_x_test, moons_y_test)

specificity: 1.0 
sensitivity: 0.9869281045751634 
accuracy: 0.9933333333333333 
ppv: 1.0 
npv: 0.9865771812080537


In [33]:
perform_kfold(decision_tree, moons_x_train, moons_y_train)

0.9914285714285714

### Random Forest on Moons1

In [34]:
random_forest = RandomForestClassifier()
random_forest.fit(moons_x_train, moons_y_train)

RandomForestClassifier()

In [35]:
print_results(random_forest, moons_x_test, moons_y_test)

specificity: 1.0 
sensitivity: 0.9869281045751634 
accuracy: 0.9933333333333333 
ppv: 1.0 
npv: 0.9865771812080537


In [36]:
perform_kfold(random_forest, moons_x_train, moons_y_train)

0.9928571428571429

### Multi Layer Perceptron on Moons1

In [37]:
perceptron_parameters = {'hidden_layer_sizes': (10,10),
                         'activation': 'relu',
                         'solver': 'adam',
                         'max_iter': 500,
                         'learning_rate': 'adaptive'}

perceptron = MLPClassifier(**perceptron_parameters)
perceptron.fit(moons_x_train, moons_y_train)

MLPClassifier(hidden_layer_sizes=(10, 10), learning_rate='adaptive',
              max_iter=500)

In [38]:
print_results(perceptron, moons_x_test, moons_y_test)

specificity: 1.0 
sensitivity: 0.9934640522875817 
accuracy: 0.9966666666666667 
ppv: 1.0 
npv: 0.9932432432432432


In [39]:
perform_kfold(perceptron, moons_x_train, moons_y_train)

0.99

# Spiral1

In [40]:
spiral_x_train, spiral_x_test, spiral_y_train, spiral_y_test = train_test_split(spiral.iloc[:, 0:2], spiral['label'], test_size=0.3, random_state=42)

### Decision Tree on Spira1

In [41]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(spiral_x_train, spiral_y_train)

DecisionTreeClassifier()

In [42]:
decision_tree.tree_.max_depth

8

In [43]:
print_results(decision_tree, spiral_x_test, spiral_y_test)

specificity: 0.9802631578947368 
sensitivity: 1.0 
accuracy: 0.99 
ppv: 0.9801324503311258 
npv: 1.0


In [44]:
perform_kfold(decision_tree, spiral_x_train, spiral_y_train)

0.9771428571428572

### Random Forest on Spiral1

In [45]:
random_forest = RandomForestClassifier()
random_forest.fit(spiral_x_train, spiral_y_train)

RandomForestClassifier()

In [46]:
print_results(random_forest, spiral_x_test, spiral_y_test)

specificity: 0.9802631578947368 
sensitivity: 0.9932432432432432 
accuracy: 0.9866666666666667 
ppv: 0.98 
npv: 0.9933333333333333


In [47]:
perform_kfold(random_forest, spiral_x_train, spiral_y_train)

0.9800000000000001

### Multi-layer Perceptron on Spiral1

In [48]:
perceptron_parameters = {'hidden_layer_sizes': (10,10),
                         'activation': 'relu',
                         'solver': 'adam',
                         'max_iter': 700,
                         'learning_rate': 'adaptive'}

perceptron = MLPClassifier(**perceptron_parameters)
perceptron.fit(spiral_x_train, spiral_y_train)

MLPClassifier(hidden_layer_sizes=(10, 10), learning_rate='adaptive',
              max_iter=700)

In [49]:
print_results(perceptron, spiral_x_test, spiral_y_test)

specificity: 0.9802631578947368 
sensitivity: 1.0 
accuracy: 0.99 
ppv: 0.9801324503311258 
npv: 1.0


In [50]:
perform_kfold(perceptron, spiral_x_train, spiral_y_train)

0.9857142857142858

# Twogaussian33

In [51]:
gaussian33_x_train, gaussian33_x_test, gaussian33_y_train, gaussian33_y_test = train_test_split(two_gaussians33.iloc[:, 0:2], two_gaussians33['label'], test_size=0.3, random_state=42)

### Decision Tree on Twogaussian33

In [52]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(gaussian33_x_train, gaussian33_y_train)

DecisionTreeClassifier()

In [53]:
decision_tree.tree_.max_depth

5

In [54]:
print_results(decision_tree, gaussian33_x_test, gaussian33_y_test)

specificity: 0.972972972972973 
sensitivity: 0.9868421052631579 
accuracy: 0.98 
ppv: 0.974025974025974 
npv: 0.9863013698630136


In [55]:
perform_kfold(decision_tree, gaussian33_x_train, gaussian33_y_train)

0.9871428571428572

### Random Forest on Twogaussian33

In [56]:
random_forest = RandomForestClassifier()
random_forest.fit(gaussian33_x_train, gaussian33_y_train)

RandomForestClassifier()

In [57]:
print_results(random_forest, gaussian33_x_test, gaussian33_y_test)

specificity: 0.9864864864864865 
sensitivity: 0.993421052631579 
accuracy: 0.99 
ppv: 0.9869281045751634 
npv: 0.9931972789115646


In [58]:
perform_kfold(random_forest, gaussian33_x_train, gaussian33_y_train)

0.9885714285714287

### Multi-layer Perceptron on Twogaussian33

In [59]:
perceptron_parameters = {'hidden_layer_sizes': (10,10),
                         'activation': 'relu',
                         'solver': 'adam',
                         'max_iter': 250,
                         'learning_rate': 'adaptive'}

perceptron = MLPClassifier(**perceptron_parameters)
perceptron.fit(gaussian33_x_train, gaussian33_y_train)

MLPClassifier(hidden_layer_sizes=(10, 10), learning_rate='adaptive',
              max_iter=250)

In [60]:
print_results(perceptron, gaussian33_x_test, gaussian33_y_test)

specificity: 0.9864864864864865 
sensitivity: 0.9868421052631579 
accuracy: 0.9866666666666667 
ppv: 0.9868421052631579 
npv: 0.9864864864864865


In [61]:
perform_kfold(perceptron, gaussian33_x_train, gaussian33_y_train)

0.9914285714285714

# Twogaussian42

In [62]:
gaussian42_x_train, gaussian42_x_test, gaussian42_y_train, gaussian42_y_test = train_test_split(two_gaussians42.iloc[:, 0:2], two_gaussians42['label'], test_size=0.3, random_state=42)

### Decision Tree on Twogaussian42

In [63]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(gaussian42_x_train, gaussian42_y_train)

DecisionTreeClassifier()

In [64]:
decision_tree.tree_.max_depth

10

In [65]:
print_results(decision_tree, gaussian42_x_test, gaussian42_y_test)

specificity: 0.9455782312925171 
sensitivity: 0.9215686274509803 
accuracy: 0.9333333333333333 
ppv: 0.9463087248322147 
npv: 0.9205298013245033


In [66]:
perform_kfold(decision_tree, gaussian42_x_train, gaussian42_y_train)

0.9128571428571428

### Random Forest on Twogaussian42

In [67]:
random_forest = RandomForestClassifier()
random_forest.fit(gaussian42_x_train, gaussian42_y_train)

RandomForestClassifier()

In [68]:
print_results(random_forest, gaussian42_x_test, gaussian42_y_test)

specificity: 0.9523809523809523 
sensitivity: 0.9150326797385621 
accuracy: 0.9333333333333333 
ppv: 0.9523809523809523 
npv: 0.9150326797385621


In [69]:
perform_kfold(random_forest, gaussian42_x_train, gaussian42_y_train)

0.9357142857142857

### Multi-layer Perceptron on Twogaussian42

In [70]:
perceptron_parameters = {'hidden_layer_sizes': (10,10),
                         'activation': 'relu',
                         'solver': 'adam',
                         'max_iter': 600,
                         'learning_rate': 'adaptive'}

perceptron = MLPClassifier(**perceptron_parameters)
perceptron.fit(gaussian42_x_train, gaussian42_y_train)

MLPClassifier(hidden_layer_sizes=(10, 10), learning_rate='adaptive',
              max_iter=600)

In [71]:
print_results(perceptron, gaussian42_x_test, gaussian42_y_test)

specificity: 0.9591836734693877 
sensitivity: 0.869281045751634 
accuracy: 0.9133333333333333 
ppv: 0.9568345323741008 
npv: 0.8757763975155279


In [72]:
perform_kfold(perceptron, gaussian42_x_train, gaussian42_y_train)

0.9414285714285715

# Breast Cancer Dataset

In [73]:
breast_cancer.head()

,STK35,DPYSL4,GJC2,FMNL1,LIG3,CA6,BRPF1,BRMS1,CXCL13,SIAE,...,ADRB1,DRD4,GABRR1,KRT10,PIP,MS4A3,SECISBP2L,EMP3,HOXB1,Class
0,0.425,-0.316,0.465,-0.142,-0.033,0.357,-0.197,-0.280,3.164,-0.086,...,-0.188,-0.015,-0.035,0.722,0.131,-0.545,0.323,-0.400,0.064,Basal
1,0.694,-0.497,0.200,-0.215,0.852,0.300,-0.864,-0.222,6.179,-0.649,...,0.200,-0.416,0.293,-0.795,1.785,-1.224,0.505,-0.275,0.300,Basal
2,0.304,-0.205,0.739,0.176,0.726,0.715,0.298,0.147,3.612,-0.230,...,0.211,0.170,0.156,-0.733,2.414,-0.435,0.047,-0.980,0.233,Basal
3,0.309,-0.561,0.669,-0.514,0.805,0.409,-0.974,0.054,1.280,-0.150,...,-0.532,0.884,0.633,-0.586,0.163,-0.987,0.223,-1.090,0.603,Basal
4,0.732,-0.287,0.311,-0.326,-0.468,1.887,-0.347,-0.193,2.410,0.479,...,0.354,0.596,0.270,-0.282,0.601,0.164,0.040,-0.705,0.054,Basal


In [74]:
breast_cancer.describe()

,STK35,DPYSL4,GJC2,FMNL1,LIG3,CA6,BRPF1,BRMS1,CXCL13,SIAE,...,MAS1,ADRB1,DRD4,GABRR1,KRT10,PIP,MS4A3,SECISBP2L,EMP3,HOXB1
count,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,...,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000
mean,0.096057,-0.186918,0.523943,-0.144778,0.314177,0.181025,-0.387842,0.036722,2.539418,0.146006,...,0.074354,-0.004728,1.015829,0.153108,-0.966392,3.626222,-0.405956,0.282146,-0.863665,0.111025
std,0.208323,0.289994,0.350318,0.345459,0.443096,0.494525,0.338736,0.424131,1.636105,0.527961,...,0.431365,0.425453,0.674723,0.221736,0.716756,2.401070,0.405931,0.388707,0.408621,0.223153
min,-0.446000,-1.064000,-0.306000,-1.155000,-0.787000,-0.791000,-1.201000,-0.855000,-0.075000,-1.123000,...,-1.015000,-1.446000,-0.745000,-0.428000,-2.926000,-0.071000,-1.796000,-0.442000,-1.722000,-0.591000
25%,-0.040000,-0.367750,0.263000,-0.366500,-0.001250,-0.054500,-0.622750,-0.266750,1.229750,-0.170500,...,-0.236250,-0.261500,0.564000,0.028000,-1.318250,1.328500,-0.627000,0.043750,-1.133000,-0.028750
50%,0.066500,-0.188000,0.471500,-0.188000,0.318000,0.119000,-0.389500,0.008500,2.280000,0.103500,...,0.083000,0.011000,0.939500,0.155000,-0.943000,3.708500,-0.388000,0.221500,-0.890500,0.103000
75%,0.215750,-0.042750,0.736000,0.037750,0.625000,0.304500,-0.192250,0.241750,3.554750,0.409500,...,0.363750,0.206750,1.403500,0.270000,-0.591000,5.589750,-0.134750,0.451000,-0.618750,0.250750
max,0.934000,0.890000,1.752000,1.870000,1.464000,3.075000,0.849000,1.502000,6.179000,2.156000,...,1.170000,1.436000,2.720000,1.169000,2.257000,8.057000,0.948000,1.888000,0.511000,0.642000


In [75]:
breast_cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Columns: 13583 entries, STK35 to Class
dtypes: float64(13582), object(1)
memory usage: 16.4+ MB


## mRMR Feature Selection

In [76]:
features = breast_cancer.drop(labels=['Class'], axis=1)
mrmr_features_list = mrmr.mrmr_classif(features, breast_cancer["Class"], 20)
selected_features = features[mrmr_features_list]

100%|██████████| 20/20 [00:30<00:00,  1.50s/it]


In [77]:
cancer_x_train, cancer_x_test, cancer_y_train, cancer_y_test = train_test_split(selected_features, breast_cancer['Class'], test_size=0.3, random_state=42)

In [78]:
svm_grid_search(cancer_x_train, cancer_y_train)

Fitting 10 folds for each of 96 candidates, totalling 960 fits


({'C': 0.1, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'},
 0.8545454545454545)

In [79]:
svm_best_classifier = SVC(C=0.1, gamma='scale', kernel='linear')
svm_best_classifier.fit(cancer_x_train, cancer_y_train)

SVC(C=0.1, kernel='linear')

In [80]:
print_results(svm_best_classifier, cancer_x_test, cancer_y_test, multi_class=True)

specificity: 0.9518728421601198 
sensitivity: 0.80995670995671 
accuracy: 0.925 
ppv: 0.8294444444444444 
npv: 0.9561240310077519


In [81]:
perform_kfold(svm_best_classifier, cancer_x_train, cancer_y_train)

0.8636363636363636

## Information Gain Feature Selection

In [82]:
selected_features = SelectKBest(mutual_info_classif, k=20).fit_transform(features, breast_cancer['Class'])

In [83]:
cancer_x_train, cancer_x_test, cancer_y_train, cancer_y_test = train_test_split(selected_features, breast_cancer['Class'], test_size=0.3, random_state=42)

In [84]:
svm_grid_search(cancer_x_train, cancer_y_train)

Fitting 10 folds for each of 96 candidates, totalling 960 fits


({'C': 0.1, 'degree': 3, 'gamma': 'auto', 'kernel': 'poly'},
 0.8636363636363636)

In [85]:
svm_best_classifier = SVC(C=0.1, degree=3, gamma='auto', kernel='poly')
svm_best_classifier.fit(cancer_x_train, cancer_y_train)

SVC(C=0.1, gamma='auto', kernel='poly')

In [86]:
print_results(svm_best_classifier, cancer_x_test, cancer_y_test, multi_class=True)

specificity: 0.9475878849339863 
sensitivity: 0.7929004329004329 
accuracy: 0.9166666666666666 
ppv: 0.7752380952380953 
npv: 0.9476718403547671


In [87]:
perform_kfold(svm_best_classifier, cancer_x_train, cancer_y_train)

0.8454545454545455